In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# 📁 Paths
data_path = r"C:\Users\sagni\Downloads\Sleep-Pattern-Detection\Sleep_health_and_lifestyle_dataset.csv"
model_save_path = r"C:\Users\sagni\Downloads\Sleep-Pattern-Detection\sleep_quality_model.h5"

# 📦 Load dataset
df = pd.read_csv(data_path)
print("✅ Dataset loaded. Shape:", df.shape)

# 🧹 Preprocessing
le_gender = LabelEncoder()
df['Gender'] = le_gender.fit_transform(df['Gender'])
le_bmi = LabelEncoder()
df['BMI Category'] = le_bmi.fit_transform(df['BMI Category'])

# Extract Systolic & Diastolic from 'Blood Pressure'
df[['Systolic', 'Diastolic']] = df['Blood Pressure'].str.split('/', expand=True).astype(int)
df.drop(['Person ID', 'Occupation', 'Blood Pressure', 'Sleep Disorder'], axis=1, inplace=True)

# Features and target
X = df.drop('Quality of Sleep', axis=1).values
y = df['Quality of Sleep'].values

# 🔀 Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ⚖️ Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 🤖 Build Neural Network (Regression)
model = Sequential([
    Dense(64, input_dim=X_train.shape[1], activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='linear')  # Linear output for regression
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])
print("🧠 Model summary:")
model.summary()

# 🏋️‍♂️ Train
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
                    epochs=100, batch_size=16, callbacks=[early_stop], verbose=1)

# 📊 Evaluate
loss, mae = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"✅ Test Mean Absolute Error (MAE): {mae:.2f}")

# 💾 Save model
model.save(model_save_path)
print(f"💾 Model saved to {model_save_path}")


✅ Dataset loaded. Shape: (374, 13)
🧠 Model summary:


C:\Users\sagni\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,817 (11.00 KB)

 Trainable params: 2,817 (11.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 49.9269 - mae: 6.9513 - val_loss: 36.2664 - val_mae: 5.8723
Epoch 2/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 36.0339 - mae: 5.8231 - val_loss: 21.9600 - val_mae: 4.4495
Epoch 3/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 23.2087 - mae: 4.5067 - val_loss: 10.0351 - val_mae: 2.8403
Epoch 4/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 10.8180 - mae: 2.8521 - val_loss: 4.0354 - val_mae: 1.5968
Epoch 5/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 5.6686 - mae: 1.9536 - val_loss: 2.5669 - val_mae: 1.0991
Epoch 6/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.7699 - mae: 1.7827 - val_loss: 1.9363 - val_mae: 0.9821
Epoch 7/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.5655 - mae: 1.7143 - val_loss: 1.5679 - val_mae: 0.8467
Epoch 8/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 3.3253 - mae: 1.3946 - val_loss: 1.3437 - val_mae: 0.7727
Epoch 9/100
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - los

✅ Test Mean Absolute Error (MAE): 0.13
💾 Model saved to C:\Users\sagni\Downloads\Sleep-Pattern-Detection\sleep_quality_model.h5
